# Multipool Google Sentiment Analysis

this function will get data from google_result table
and calculate the sentiment analysis. Result is store in google_result_sentiment

crontab command
* * * * * /home/haviz/multipool/ai-case-study/run-ai-sa-google.sh >> /home/haviz/multipool/ai-sa-google.log



In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import pymysql.cursors
import sqlalchemy as sa
import nltk
import string
import os

from nltk.corpus import stopwords 
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
import pandas as pd

# nltk.download('stopwords')


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")

In [18]:
from sqlalchemy.util import deprecations
deprecations.SILENCE_UBER_WARNING = True

def execute_query(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '98.98.117.105',
        'port': '5432',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData()

    # Example: Define your table
    your_table = Table('your_table', metadata,
                       Column('id', Integer, primary_key=True),
                       Column('column1', String),
                       Column('column2', String)
                       )

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()

In [11]:
import pandas as pd
import re
from sqlalchemy import create_engine

def stemming(comment):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in comment:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    return d_clean
    
# function case folding
def casefolding(comment):
    comment = comment.lower()
    comment = comment.strip(" ")
    comment = re.sub(r'[?|$|.|!_:")(-+,]','',comment)
    return comment

def clean_up_tag(comment):
    x_ret = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",comment).split())
    return x_ret

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    
    # remove whitspace
    strOut = strOut.strip()
    
    # 
    strOut = re.sub('\s+',' ',strOut)
    return strOut

In [12]:
def process_by_df(source_ds, col_name):
    from tqdm import tqdm
    pd.options.mode.chained_assignment = None 
    
    # trim to 2500 character
    source_ds[col_name] = source_ds[col_name].str.slice(0,2000)

    # skip stemming
    source_ds['stemmed'] = source_ds[col_name].apply(clean_up_tag)
    source_ds['stemmed'] = source_ds['stemmed'].apply(casefolding)
    source_ds['stemmed'] = source_ds['stemmed'].apply(text_preproc)

    sw = stopwords.words('indonesian')
    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)
    sw = stopwords.words('indonesian')

    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay

    # Create a new list and insert each element from the original list
    from tqdm import tqdm

    list_text = source_ds['stemmed'].tolist()
    new_list = []
    for i in tqdm(range( len(list_text) )):
        # print(len(list_text[i]))
        try:
            if len(list_text[i]) > 0:
                res = pipe( list_text[i] )
                new_list.append(res)
            else:
                res = pipe( 'kalimat netral' )
                new_list.append(res)
                zip
        except Exception as e:
            # Handle the exception (error) here
            print(f"Error processing value at index {i}: {list_text[i]}")
            print(f"Error message: {str(e)}")
            res = pipe( 'kalimat netral' )
            new_list.append(res)      
            
            continue

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay
    # create a list of our conditions
    for i in range(0, len(new_list)):
        # print(new_list[i][0]['label'])
        if new_list[i][0]['label'] == 'Positive':
            new_list[i][0]['Value'] = 1
        elif new_list[i][0]['label'] == 'Negative':
            new_list[i][0]['Value'] = -1
        elif new_list[i][0]['label'] == 'Neutral':
            new_list[i][0]['Value'] = 0

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']
        source_ds.at[index,'Score'] = new_list[index][0]['score']
        #source_ds.at[index,'Score'] = new_list[index][0]['Value']
        
    return source_ds


In [19]:
# query 400 rows of data query data from table
select_query = "select * \
from news_national_medol gr \
where id not in (select news_national_medol_id from news_national_medol_sentiment) \
order by id desc limit 500"

source_ds = execute_query(select_query)

if len(source_ds) == 0:
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()
    
# source_ds.head()

In [20]:
ds_title = source_ds
res_ds = process_by_df(ds_title,'title')

100%|██████████| 500/500 [00:15<00:00, 31.60it/s]


In [21]:
res_ds.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,national_medol_id,is_polri,reporter,editor,is_deleted,stemmed,tokenized_tweet,Prediction,Score
0,22063,"Pernah Viral Gagal Jambret HP Bocah, AF Kini P...",BEKASI - Pemuda berinisial AF (28) kini menge...,"Sabtu, 09 Maret 2024 - 00:12 WIB",,https://index.sindonews.com/,https://metro.sindonews.com/read/1336755/170/p...,2024-03-09 07:24:18.577745,20,False,,Ade Suhardi,False,pernah viral gagal jambret hp bocah af kini pa...,"{af, viral, baju, pakai, gagal, babelan, tahan...",Neutral,0.999716
1,22062,Juergen Klopp Bongkar Rahasia Darwin Nunez: Di...,"Penyerang Liverpool, Darwin Nunez , memiliki ...","Sabtu, 09 Maret 2024 - 01:01 WIB",,https://index.sindonews.com/,https://sports.sindonews.com/read/1336189/11/j...,2024-03-09 07:24:18.570927,20,False,,Andika Rachmansyah,False,juergen klopp bongkar rahasia darwin nunez dia...,"{klopp, ancaman, nunez, bongkar, rahasia, juer...",Neutral,0.999623
2,22061,"Jadi Takjil Favorit Ramadan, Ini Perbedaan Es ...",JAKARTA - Berburu takjil Ramadan menjadi kegi...,"Sabtu, 09 Maret 2024 - 03:30 WIB",,https://index.sindonews.com/,https://lifestyle.sindonews.com/read/1336595/1...,2024-03-09 07:24:18.567557,20,False,,Lutfan Faizi,False,jadi takjil favorit ramadan ini perbedaan es b...,"{es, perbedaan, sop, favorit, ramadan, buah, t...",Positive,0.982308
3,22060,Ini 5 Petinju Dunia yang Jadi Teman Dekat Cris...,Sedikitnya ada lima petinju dunia yang menjad...,"Sabtu, 09 Maret 2024 - 04:02 WIB",,https://index.sindonews.com/,https://sports.sindonews.com/read/1336263/50/i...,2024-03-09 07:24:18.564039,20,False,,Lutfan Faizi,False,ini petinju dunia yang jadi teman dekat cristi...,"{dunia, petinju, ronaldo, teman, cristiano}",Neutral,0.996944
4,22059,4 Jenis Takjil yang Harus Dihindari Penderita ...,JAKARTA - Terdapat sedikitnya empat jenis tak...,"Sabtu, 09 Maret 2024 - 04:40 WIB",,https://index.sindonews.com/,https://lifestyle.sindonews.com/read/1336749/1...,2024-03-09 07:24:18.560126,20,False,,Lutfan Faizi,False,jenis takjil yang harus dihindari penderita di...,"{penderita, diabetes, dihindari, jenis, takjil}",Neutral,0.997442


In [22]:
# record result
sentiment_class = 0

# INSERT INTO public.news_national_medol_sentiment (sentiment_category, score, news_national_medol_id) VALUES( 0, 0, 0);

for index, row in res_ds.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    sql = "INSERT INTO news_national_medol_sentiment (sentiment_category, score, news_national_medol_id) VALUES(%s, %s, %s);" % (sentiment_class,row['Score'],row['id']) 
    # print(sql)
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 row

In [23]:
ds_content = source_ds
res_ds_content = process_by_df(ds_content,'content')

100%|██████████| 500/500 [00:57<00:00,  8.69it/s]


In [24]:
res_ds_content.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,national_medol_id,is_polri,reporter,editor,is_deleted,stemmed,tokenized_tweet,Prediction,Score
0,22063,"Pernah Viral Gagal Jambret HP Bocah, AF Kini P...",BEKASI - Pemuda berinisial AF (28) kini menge...,"Sabtu, 09 Maret 2024 - 00:12 WIB",,https://index.sindonews.com/,https://metro.sindonews.com/read/1336755/170/p...,2024-03-09 07:24:18.577745,20,False,,Ade Suhardi,False,bekasi pemuda berinisial af kini mengenakan ba...,"{sosial, mn, polisi, pergi, warga, pelaku, per...",Neutral,0.517755
1,22062,Juergen Klopp Bongkar Rahasia Darwin Nunez: Di...,"Penyerang Liverpool, Darwin Nunez , memiliki ...","Sabtu, 09 Maret 2024 - 01:01 WIB",,https://index.sindonews.com/,https://sports.sindonews.com/read/1336189/11/j...,2024-03-09 07:24:18.570927,20,False,,Andika Rachmansyah,False,penyerang liverpool darwin nunez memiliki andi...,"{absolutnya, terangnya, the, mencetak, final, ...",Positive,0.996638
2,22061,"Jadi Takjil Favorit Ramadan, Ini Perbedaan Es ...",JAKARTA - Berburu takjil Ramadan menjadi kegi...,"Sabtu, 09 Maret 2024 - 03:30 WIB",,https://index.sindonews.com/,https://lifestyle.sindonews.com/read/1336595/1...,2024-03-09 07:24:18.567557,20,False,,Lutfan Faizi,False,jakarta berburu takjil ramadan menjadi kegiata...,"{cenderung, dingin, disajikan, rebusan, salah,...",Positive,0.993043
3,22060,Ini 5 Petinju Dunia yang Jadi Teman Dekat Cris...,Sedikitnya ada lima petinju dunia yang menjad...,"Sabtu, 09 Maret 2024 - 04:02 WIB",,https://index.sindonews.com/,https://sports.sindonews.com/read/1336263/50/i...,2024-03-09 07:24:18.564039,20,False,,Lutfan Faizi,False,sedikitnya ada lima petinju dunia yang menjadi...,"{petinju, dazn, mengutip, salah, furytys, jr, ...",Positive,0.976718
4,22059,4 Jenis Takjil yang Harus Dihindari Penderita ...,JAKARTA - Terdapat sedikitnya empat jenis tak...,"Sabtu, 09 Maret 2024 - 04:40 WIB",,https://index.sindonews.com/,https://lifestyle.sindonews.com/read/1336749/1...,2024-03-09 07:24:18.560126,20,False,,Lutfan Faizi,False,jakarta terdapat sedikitnya empat jenis takjil...,"{buatan, dingin, umat, gorenganmasyarakat, kew...",Positive,0.993362


In [ ]:
# record result
sentiment_class = 0

for index, row in res_ds_content.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    sql = "update news_national_medol_sentiment set sentiment_category_content = %s, score_content = %s where news_national_medol_id = %s" % (sentiment_class,row['Score'],row['id']) 
    # print(sql)
    execute_query(sql)